In [18]:
import pandas as pd
import boto3
import numpy as np

In [19]:
bucket = 'datascience-sagemaker-talitasantos-ac5'

In [20]:
dados_brutos_arquivo = 'Brasileirao_Matches.csv'

try:
    os.mkdir('dados')
except:
    pass

dados = pd.read_csv("s3://{}/{}".format(bucket, "Brasileirao_Matches.csv"))
pd.set_option('display.max_rows', 20) 
dados

,datetime,home_team,home_team_state,away_team,away_team_state,home_goal,away_goal,season,round
0,2012-05-19 18:30:00,Palmeiras,SP,Portuguesa,SP,1.0,1.0,2012,1
1,2012-05-19 18:30:00,Sport,PE,Flamengo,RJ,1.0,1.0,2012,1
2,2012-05-19 21:00:00,Figueirense,SC,Náutico,PE,2.0,1.0,2012,1
3,2012-05-20 16:00:00,Ponte Preta,SP,Atlético,MG,0.0,1.0,2012,1
4,2012-05-20 16:00:00,Corinthians,SP,Fluminense,RJ,0.0,1.0,2012,1
...,...,...,...,...,...,...,...,...,...
3795,2021-12-09 21:30:00,Fortaleza,CE,Bahia,BA,NaN,NaN,2021,38
3796,2021-12-09 21:30:00,Sport,PE,Athletico Paranaense,PR,NaN,NaN,2021,38
3797,2021-12-09 21:30:00,Red Bull Bragantino,SP,Internacional,RS,NaN,NaN,2021,38
3798,2021-12-09 21:30:00,Juventude,RS,Corinthians,SP,NaN,NaN,2021,38


In [21]:
dados = dados.drop(['home_team_state', 'away_team_state','round'], axis=1)

In [22]:
for x in dados:
  stage = 'pontos corridos'
  dados['stage'] = stage
dados.columns 

dados['home_goal']=dados['home_goal'].fillna('0')
dados['away_goal']=dados['away_goal'].fillna('0')

In [23]:
dados_brutos_arquivo = 'Libertadores_Matches.csv'

try:
    os.mkdir('dados1')
except:
    pass

dados1 = pd.read_csv("s3://{}/{}".format(bucket, "Libertadores_Matches.csv"))
pd.set_option('display.max_rows', 20) 
dados1

,datetime,home_team,away_team,home_goal,away_goal,season,stage
0,2013-02-12 20:15:00,Nacional (URU),Barcelona-EQU,2,2,2013.0,group stage
1,2013-02-13 22:00:00,Boca Juniors,Toluca,1,2,2013.0,group stage
2,2013-02-19 23:45:00,Toluca,Nacional (URU),2,3,2013.0,group stage
3,2013-02-27 19:45:00,Barcelona-EQU,Boca Juniors,1,2,2013.0,group stage
4,2013-03-07 00:15:00,Toluca,Barcelona-EQU,1,1,2013.0,group stage
...,...,...,...,...,...,...,...
1126,2021-09-22 21:30:00,Flamengo,Barcelona-EQU,2,0,2021.0,semifinals
1127,2021-09-29 21:30:00,Barcelona-EQU,Flamengo,0,2,2021.0,semifinals
1128,2021-09-21 21:30:00,Palmeiras,Atlético-MG,0,0,2021.0,semifinals
1129,2021-09-28 21:30:00,Atlético-MG,Palmeiras,1,1,2021.0,semifinals


In [24]:
dados1['home_goal'] = dados1['home_goal'].str.replace('-','0')
dados1['away_goal'] = dados1['away_goal'].str.replace('-','0')

In [25]:
dataset = pd.concat([dados, dados1])
dataset

,datetime,home_team,away_team,home_goal,away_goal,season,stage
0,2012-05-19 18:30:00,Palmeiras,Portuguesa,1,1,2012.0,pontos corridos
1,2012-05-19 18:30:00,Sport,Flamengo,1,1,2012.0,pontos corridos
2,2012-05-19 21:00:00,Figueirense,Náutico,2,1,2012.0,pontos corridos
3,2012-05-20 16:00:00,Ponte Preta,Atlético,0,1,2012.0,pontos corridos
4,2012-05-20 16:00:00,Corinthians,Fluminense,0,1,2012.0,pontos corridos
...,...,...,...,...,...,...,...
1126,2021-09-22 21:30:00,Flamengo,Barcelona-EQU,2,0,2021.0,semifinals
1127,2021-09-29 21:30:00,Barcelona-EQU,Flamengo,0,2,2021.0,semifinals
1128,2021-09-21 21:30:00,Palmeiras,Atlético-MG,0,0,2021.0,semifinals
1129,2021-09-28 21:30:00,Atlético-MG,Palmeiras,1,1,2021.0,semifinals


In [26]:
dataset['home_goal'] = dataset['home_goal'].astype('float')
dataset['away_goal'] = dataset['away_goal'].astype('float')

In [27]:
dataset['ALVO'] = dataset.apply(lambda x: 2 if x['home_goal'] > x['away_goal'] else (0 if x['home_goal'] < x['away_goal'] else 1), axis=1)

In [28]:
dataset

,datetime,home_team,away_team,home_goal,away_goal,season,stage,ALVO
0,2012-05-19 18:30:00,Palmeiras,Portuguesa,1.0,1.0,2012.0,pontos corridos,1
1,2012-05-19 18:30:00,Sport,Flamengo,1.0,1.0,2012.0,pontos corridos,1
2,2012-05-19 21:00:00,Figueirense,Náutico,2.0,1.0,2012.0,pontos corridos,2
3,2012-05-20 16:00:00,Ponte Preta,Atlético,0.0,1.0,2012.0,pontos corridos,0
4,2012-05-20 16:00:00,Corinthians,Fluminense,0.0,1.0,2012.0,pontos corridos,0
...,...,...,...,...,...,...,...,...
1126,2021-09-22 21:30:00,Flamengo,Barcelona-EQU,2.0,0.0,2021.0,semifinals,2
1127,2021-09-29 21:30:00,Barcelona-EQU,Flamengo,0.0,2.0,2021.0,semifinals,0
1128,2021-09-21 21:30:00,Palmeiras,Atlético-MG,0.0,0.0,2021.0,semifinals,1
1129,2021-09-28 21:30:00,Atlético-MG,Palmeiras,1.0,1.0,2021.0,semifinals,1


In [29]:
dataset['Pre_Home_Palmeiras'] = dataset['home_team'].apply(lambda x: 1 if x=='Palmeiras' else 0)
dataset['Pre_Home_Santos'] = dataset['home_team'].apply(lambda x: 1 if x=='Santos' else 0)
dataset['Pre_Home_SaoPaulo'] = dataset['home_team'].apply(lambda x: 1 if x=='São Paulo' else 0)
dataset['Pre_Home_Corinthians'] = dataset['home_team'].apply(lambda x: 1 if x=='Corinthians' else 0)

dataset['Pre_Away_Palmeiras'] = dataset['away_team'].apply(lambda x: 1 if x=='Palmeiras' else 0)
dataset['Pre_Away_Santos'] = dataset['away_team'].apply(lambda x: 1 if x=='Santos' else 0)
dataset['Pre_Away_SaoPaulo'] = dataset['away_team'].apply(lambda x: 1 if x=='São Paulo' else 0)
dataset['Pre_Away_Corinthians'] = dataset['away_team'].apply(lambda x: 1 if x=='Corinthians' else 0)

#normalização
dataset['Pre_home_goal'] = [1. if x>8 else (x-0)/(8-0) for x in dataset['home_goal']]
dataset['Pre_away_goal'] = [1. if x>7 else (x-0)/(7-0) for x in dataset['away_goal']]
dataset

,datetime,home_team,away_team,home_goal,away_goal,season,stage,ALVO,Pre_Home_Palmeiras,Pre_Home_Santos,Pre_Home_SaoPaulo,Pre_Home_Corinthians,Pre_Away_Palmeiras,Pre_Away_Santos,Pre_Away_SaoPaulo,Pre_Away_Corinthians,Pre_home_goal,Pre_away_goal
0,2012-05-19 18:30:00,Palmeiras,Portuguesa,1.0,1.0,2012.0,pontos corridos,1,1,0,0,0,0,0,0,0,0.125,0.142857
1,2012-05-19 18:30:00,Sport,Flamengo,1.0,1.0,2012.0,pontos corridos,1,0,0,0,0,0,0,0,0,0.125,0.142857
2,2012-05-19 21:00:00,Figueirense,Náutico,2.0,1.0,2012.0,pontos corridos,2,0,0,0,0,0,0,0,0,0.250,0.142857
3,2012-05-20 16:00:00,Ponte Preta,Atlético,0.0,1.0,2012.0,pontos corridos,0,0,0,0,0,0,0,0,0,0.000,0.142857
4,2012-05-20 16:00:00,Corinthians,Fluminense,0.0,1.0,2012.0,pontos corridos,0,0,0,0,1,0,0,0,0,0.000,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126,2021-09-22 21:30:00,Flamengo,Barcelona-EQU,2.0,0.0,2021.0,semifinals,2,0,0,0,0,0,0,0,0,0.250,0.000000
1127,2021-09-29 21:30:00,Barcelona-EQU,Flamengo,0.0,2.0,2021.0,semifinals,0,0,0,0,0,0,0,0,0,0.000,0.285714
1128,2021-09-21 21:30:00,Palmeiras,Atlético-MG,0.0,0.0,2021.0,semifinals,1,1,0,0,0,0,0,0,0,0.000,0.000000
1129,2021-09-28 21:30:00,Atlético-MG,Palmeiras,1.0,1.0,2021.0,semifinals,1,0,0,0,0,1,0,0,0,0.125,0.142857


In [30]:
# atributos
X = dataset[['home_goal', 'away_goal',
     'Pre_Home_Palmeiras', 'Pre_Home_Santos',
       'Pre_Home_SaoPaulo', 'Pre_Home_Corinthians', 'Pre_Away_Palmeiras',
       'Pre_Away_Santos', 'Pre_Away_SaoPaulo', 'Pre_Away_Corinthians',
       'Pre_home_goal', 'Pre_away_goal']]

# categoria
y = dataset['ALVO']

In [45]:
X

,home_goal,away_goal,Pre_Home_Palmeiras,Pre_Home_Santos,Pre_Home_SaoPaulo,Pre_Home_Corinthians,Pre_Away_Palmeiras,Pre_Away_Santos,Pre_Away_SaoPaulo,Pre_Away_Corinthians,Pre_home_goal,Pre_away_goal
0,1.0,1.0,1,0,0,0,0,0,0,0,0.125,0.142857
1,1.0,1.0,0,0,0,0,0,0,0,0,0.125,0.142857
2,2.0,1.0,0,0,0,0,0,0,0,0,0.250,0.142857
3,0.0,1.0,0,0,0,0,0,0,0,0,0.000,0.142857
4,0.0,1.0,0,0,0,1,0,0,0,0,0.000,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...
1126,2.0,0.0,0,0,0,0,0,0,0,0,0.250,0.000000
1127,0.0,2.0,0,0,0,0,0,0,0,0,0.000,0.285714
1128,0.0,0.0,1,0,0,0,0,0,0,0,0.000,0.000000
1129,1.0,1.0,0,0,0,0,1,0,0,0,0.125,0.142857


In [31]:
treinamento, validacao, teste = np.split(dataset.sample(frac=1, random_state=88), [int(0.7 * len(dataset)), int(0.9 * len(dataset))]) 
len(dataset),len(treinamento), len(validacao), len(teste)

(4931, 3451, 986, 494)

In [33]:
treinamento.to_csv('treinamento.csv', index=False, header=False)
validacao.to_csv('validacao.csv', index=False, header=False)
teste.to_csv('teste.csv', index=False)

In [34]:
boto3.Session().resource('s3').Bucket(bucket).Object('treinamento.csv').upload_file('treinamento.csv')
boto3.Session().resource('s3').Bucket(bucket).Object('validacao.csv').upload_file('validacao.csv')
boto3.Session().resource('s3').Bucket(bucket).Object('teste.csv').upload_file('teste.csv')

In [36]:
import sagemaker
from sagemaker.session import TrainingInput
import numpy as np
from sagemaker.pytorch import PyTorch
import pandas as pd
from sklearn.metrics import classification_report

sagemaker_session = sagemaker.Session()

bucket = 'datascience-sagemaker-talitasantos-ac5'
prefix = 'mlp/campeonato'

instance_type = 'ml.m5.large'

role = sagemaker.get_execution_role()

In [37]:
#salvar arquivo csv
dataset.to_csv('data/train.csv')

# enviar para s3
sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)

's3://datascience-sagemaker-talitasantos-ac5/mlp/campeonato'

In [38]:
train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train.csv"), content_type="csv"
)

In [46]:
estimator = PyTorch(entry_point='mlp.py',
                    base_name='campeonato_MLP',
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters={
                        'epochs': 10,
                    })

In [ ]:
estimator.fit({"training": train_input})

2022-05-25 01:21:13 Starting - Starting the training job...
2022-05-25 01:21:43 Starting - Preparing the instances for trainingProfilerReport-1653441673: InProgress
.........
2022-05-25 01:23:04 Downloading - Downloading input data....